In [ ]:
!pip install pandas

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import json
import random
import cv2 # Sử dụng cv2 để đọc ảnh
import shutil # Để sao chép file
from glob import glob
from tqdm import tqdm
from PIL import Image # Dùng PIL để đọc ảnh cho Faster R-CNN dataset

# Cài đặt các thư viện cần thiết
# Lưu ý: Cài đặt này có thể mất một chút thời gian
!pip install -q cython
!pip install -q pycocotools
!pip install -q opencv-python-headless # Để tránh lỗi GUI trên Kaggle
!pip install -q torch torchvision # Đảm bảo có sẵn nếu chưa

# --- 1. Thiết lập đường dẫn và cấu hình ---

# Đường dẫn gốc của dataset YOLO ban đầu trên Kaggle
# ĐẢM BẢO TÊN DATASET TRÊN KAGGLE CỦA BẠN CHÍNH XÁC LÀ "vietnamses-traffic-sign-detection-augmentaion"
DATA_DIR = "/kaggle/input/vietnamses-traffic-sign-detection-augmentaion/dataset"
CLASSES_FILE = os.path.join(DATA_DIR, 'classes.txt')

# Đường dẫn đầu ra cho dataset COCO đã chuyển đổi (phải nằm trong /kaggle/working/)
OUTPUT_DIR = '/kaggle/working/faster_rcnn_coco_dataset' # Đổi tên cho rõ ràng hơn
OUTPUT_ANNOTATIONS_DIR = os.path.join(OUTPUT_DIR, 'annotations')
OUTPUT_IMAGES_DIR = os.path.join(OUTPUT_DIR, 'images')

# Tạo các thư mục đầu ra nếu chưa tồn tại
os.makedirs(OUTPUT_ANNOTATIONS_DIR, exist_ok=True)
os.makedirs(os.path.join(OUTPUT_IMAGES_DIR, 'train'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_IMAGES_DIR, 'val'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_IMAGES_DIR, 'test'), exist_ok=True)

# Đường dẫn các file JSON COCO đầu ra
TRAIN_JSON_PATH = os.path.join(OUTPUT_ANNOTATIONS_DIR, 'instances_train.json')
VAL_JSON_PATH = os.path.join(OUTPUT_ANNOTATIONS_DIR, 'instances_val.json')
TEST_JSON_PATH = os.path.join(OUTPUT_ANNOTATIONS_DIR, 'instances_test.json')

# Đọc tên các lớp từ file classes.txt
class_names = []
try:
    with open(CLASSES_FILE, 'r') as f:
        for line in f:
            class_names.append(line.strip())
    print(f"Đã đọc {len(class_names)} tên lớp từ {CLASSES_FILE}:")
    print(class_names)
except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file {CLASSES_FILE}. Vui lòng kiểm tra lại đường dẫn và đảm bảo dataset đã được thêm.")
    raise # Dừng chương trình nếu không tìm thấy file quan trọng này
except Exception as e:
    print(f"Lỗi khi đọc file classes.txt: {e}")
    raise

# Định nghĩa category_id cho COCO (bắt đầu từ 1, vì COCO có class 0 là background)
coco_categories = [{"id": i + 1, "name": name, "supercategory": "traffic_sign"}
                   for i, name in enumerate(class_names)]
num_classes = len(class_names) # Số lượng lớp (không bao gồm background)

print(f"Tổng số lớp đối tượng: {num_classes}")

# --- 2. Hàm chuyển đổi YOLO sang COCO ---

def yolo_to_coco_json(data_pairs, split_name, image_output_dir, 
                      start_image_id, start_annotation_id, categories):
    """
    Chuyển đổi các cặp tệp ảnh-nhãn YOLO sang định dạng COCO JSON.

    Args:
        data_pairs (list): Danh sách các tuple (full_image_path, full_label_path).
        split_name (str): Tên của tập dữ liệu ('train', 'val', 'test').
        image_output_dir (str): Thư mục đích để sao chép ảnh vào (ví dụ: /kaggle/working/coco_dataset/images/train).
        start_image_id (int): ID bắt đầu cho ảnh trong JSON COCO.
        start_annotation_id (int): ID bắt đầu cho annotation trong JSON COCO.
        categories (list): Danh sách các dictionary category COCO.

    Returns:
        dict: Dictionary COCO JSON.
        int: ID ảnh tiếp theo.
        int: ID annotation tiếp theo.
    """
    coco_images = []
    coco_annotations = []
    current_image_id = start_image_id
    current_annotation_id = start_annotation_id

    print(f"Bắt đầu xử lý {len(data_pairs)} cặp ảnh-nhãn cho tập {split_name}...")

    for img_full_path, label_full_path in tqdm(data_pairs, desc=f"Chuyển đổi {split_name}"):
        img_name = os.path.basename(img_full_path) # Lấy chỉ tên file ảnh
        
        # Sao chép ảnh sang thư mục COCO output
        dest_image_path = os.path.join(image_output_dir, img_name)
        shutil.copy(img_full_path, dest_image_path)

        # Đọc ảnh để lấy kích thước
        try:
            img = Image.open(img_full_path).convert("RGB")
            width, height = img.size
        except Exception as e:
            print(f"Cảnh báo: Không thể đọc ảnh {img_full_path}. Bỏ qua. Lỗi: {e}")
            continue

        image_info = {
            "id": current_image_id,
            "file_name": os.path.join(split_name, img_name), # Đường dẫn tương đối trong COCO JSON
            "width": width,
            "height": height
        }
        coco_images.append(image_info)

        # Đọc nhãn từ đường dẫn nhãn chính xác đã được truyền vào
        # Chúng ta đã kiểm tra sự tồn tại của file nhãn khi thu thập các cặp
        with open(label_full_path, 'r') as f: 
            for line in f:
                parts = line.strip().split()
                if len(parts) != 5:
                    print(f"Cảnh báo: Định dạng nhãn không hợp lệ tại {label_full_path}, dòng: '{line.strip()}'. Bỏ qua.")
                    continue

                class_idx = int(float(parts[0]))
                x_center, y_center, w_norm, h_norm = map(float, parts[1:])

                # Chuyển đổi từ normalized center_x, center_y, width, height sang x_min, y_min, width, height (pixel)
                x_min = (x_center - w_norm / 2) * width
                y_min = (y_center - h_norm / 2) * height
                box_width = w_norm * width
                box_height = h_norm * height

                # Đảm bảo các giá trị không âm và trong giới hạn ảnh
                x_min = max(0.0, x_min)
                y_min = max(0.0, y_min)
                box_width = min(width - x_min, box_width)
                box_height = min(height - y_min, box_height)
                
                # Làm tròn để tránh lỗi dấu phẩy động nhỏ
                x_min = round(x_min, 2)
                y_min = round(y_min, 2)
                box_width = round(box_width, 2)
                box_height = round(box_height, 2)

                # COCO requires category_id to be 1-indexed (class_idx từ YOLO là 0-indexed)
                category_id = class_idx + 1

                if box_width > 0 and box_height > 0: # Chỉ thêm annotation nếu box hợp lệ
                    annotation_info = {
                        "id": current_annotation_id,
                        "image_id": current_image_id,
                        "category_id": category_id,
                        "bbox": [x_min, y_min, box_width, box_height],
                        "area": round(box_width * box_height, 2),
                        "iscrowd": 0
                    }
                    coco_annotations.append(annotation_info)
                    current_annotation_id += 1
                else:
                    print(f"Cảnh báo: Bounding box không hợp lệ (width <= 0 hoặc height <= 0) cho ảnh {img_name}, nhãn {label_full_path}. Bỏ qua annotation này.")

        current_image_id += 1

    coco_json = {
        "info": {"description": f"COCO format dataset for {split_name} split"},
        "licenses": [],
        "images": coco_images,
        "annotations": coco_annotations,
        "categories": categories
    }
    return coco_json, current_image_id, current_annotation_id

# --- 3. Thực hiện chuyển đổi cho từng tập gốc (Train, Val, Test) ---

print("\n--- Bắt đầu chuyển đổi dataset YOLO sang COCO theo các tập gốc ---")

# Đường dẫn các thư mục ảnh và nhãn gốc của dataset YOLO
SRC_SPLITS = {
    'train': {
        'images': os.path.join(DATA_DIR, 'images', 'train'),
        'labels': os.path.join(DATA_DIR, 'labels', 'train'),
        'output_json': TRAIN_JSON_PATH,
        'output_images_dir': os.path.join(OUTPUT_IMAGES_DIR, 'train')
    },
    'val': {
        'images': os.path.join(DATA_DIR, 'images', 'val'),
        'labels': os.path.join(DATA_DIR, 'labels', 'val'),
        'output_json': VAL_JSON_PATH,
        'output_images_dir': os.path.join(OUTPUT_IMAGES_DIR, 'val')
    },
    'test': {
        'images': os.path.join(DATA_DIR, 'images', 'test'),
        'labels': os.path.join(DATA_DIR, 'labels', 'test'),
        'output_json': TEST_JSON_PATH,
        'output_images_dir': os.path.join(OUTPUT_IMAGES_DIR, 'test')
    }
}

global_image_id = 0
global_annotation_id = 0

for split_name, paths in SRC_SPLITS.items():
    img_src_dir = paths['images']
    label_src_dir = paths['labels']
    output_json_path = paths['output_json']
    output_images_dir = paths['output_images_dir']

    data_pairs_for_split = []
    
    if os.path.exists(img_src_dir) and os.path.exists(label_src_dir):
        print(f"\nĐang thu thập và chuyển đổi tập '{split_name}' từ: {img_src_dir}")
        current_images = [f for f in os.listdir(img_src_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
        for img_name in current_images:
            base_name = os.path.splitext(img_name)[0]
            img_full_path = os.path.join(img_src_dir, img_name)
            label_full_path = os.path.join(label_src_dir, base_name + ".txt")
            
            if os.path.exists(label_full_path):
                data_pairs_for_split.append((img_full_path, label_full_path))
            else:
                print(f"Cảnh báo: Không tìm thấy file nhãn {label_full_path} cho ảnh {img_full_path} trong tập '{split_name}'. Ảnh này sẽ bị bỏ qua hoặc không có annotations.")
    else:
        print(f"\nCảnh báo: Thư mục ảnh hoặc nhãn gốc không tồn tại cho tập '{split_name}': {img_src_dir} hoặc {label_src_dir}. Bỏ qua việc xử lý tập này.")
        continue # Bỏ qua split này nếu thư mục không tồn tại

    if not data_pairs_for_split:
        print(f"Không có cặp ảnh-nhãn nào được tìm thấy cho tập '{split_name}'. Bỏ qua việc tạo JSON cho tập này.")
        continue

    # Chuyển đổi tập hiện tại
    coco_json_for_split, global_image_id, global_annotation_id = yolo_to_coco_json(
        data_pairs_for_split, split_name, output_images_dir,
        global_image_id, global_annotation_id, coco_categories
    )
    with open(output_json_path, 'w') as f:
        json.dump(coco_json_for_split, f, indent=4)
    print(f"Đã lưu COCO JSON cho tập {split_name} tại: {output_json_path}")

print("\n--- Chuyển đổi dataset sang COCO hoàn tất ---")


# --- 4. Hàm kiểm tra định dạng COCO (Check Format) ---

print("\n--- Bắt đầu kiểm tra định dạng COCO ---")
from pycocotools.coco import COCO

def validate_coco_json(json_path):
    """
    Kiểm tra tính hợp lệ của file JSON theo định dạng COCO.
    """
    print(f"Kiểm tra file: {json_path}")
    try:
        if not os.path.exists(json_path):
            print(f"LỖI: File JSON không tồn tại tại {json_path}.")
            return False
        
        with open(json_path, 'r') as f:
            data = json.load(f)

        # Kiểm tra các khóa cơ bản
        required_keys = ['images', 'annotations', 'categories']
        for key in required_keys:
            if key not in data:
                print(f"LỖI: '{key}' không có trong file JSON.")
                return False

        # Kiểm tra số lượng ảnh và annotations
        print(f"Số lượng ảnh: {len(data['images'])}")
        print(f"Số lượng annotations: {len(data['annotations'])}")

        if not data['images']:
            print(f"Cảnh báo: Không có ảnh trong '{json_path}'.")
        if not data['annotations']:
            print(f"Cảnh báo: Không có annotations trong '{json_path}'.")

        # Khởi tạo đối tượng COCO (sẽ ném lỗi nếu định dạng sai nghiêm trọng)
        # Đây là bước kiểm tra chính của pycocotools
        coco = COCO(json_path)
        
        # Có thể thêm các kiểm tra phụ như:
        # Kiểm tra xem tất cả image_id trong annotations có tồn tại trong images không
        # Kiểm tra xem category_id trong annotations có tồn tại trong categories không
        
        print(f"[SUCCESS]: File '{json_path}' có vẻ đúng định dạng COCO.")
        return True
    except Exception as e:
        print(f"[ERROR]: LỖI khi kiểm tra '{json_path}': {e}")
        return False

# Gọi hàm kiểm tra định dạng cho từng file
print(f"Train JSON validation: {validate_coco_json(TRAIN_JSON_PATH)}")
print(f"Validation JSON validation: {validate_coco_json(VAL_JSON_PATH)}")
print(f"Test JSON validation: {validate_coco_json(TEST_JSON_PATH)}")

print("\n--- Kiểm tra định dạng COCO hoàn tất ---")

# --- 5. Visual kiểm tra Bounding Box (Mới) ---

print("\n--- Bắt đầu kiểm tra trực quan Bounding Box ---")

import matplotlib.pyplot as plt
import matplotlib.patches as patches

def visualize_coco_boxes(coco_json_path, image_base_dir, num_samples=3):
    """
    Hiển thị ngẫu nhiên các ảnh với bounding box từ file COCO JSON.
    """
    print(f"Đang kiểm tra trực quan từ file: {coco_json_path}")
    if not os.path.exists(coco_json_path):
        print(f"LỖI: File JSON không tồn tại tại {coco_json_path}. Không thể kiểm tra trực quan.")
        return

    coco = COCO(coco_json_path)
    
    # Lấy ngẫu nhiên vài image IDs
    img_ids = coco.getImgIds()
    if len(img_ids) == 0:
        print(f"Cảnh báo: Không có ảnh nào trong {coco_json_path} để hiển thị.")
        return
        
    random_img_ids = random.sample(img_ids, min(num_samples, len(img_ids)))

    for img_id in random_img_ids:
        img_info = coco.loadImgs(img_id)[0]
        # img_file_name là đường dẫn tương đối, ví dụ: 'train/image_0001.jpg'
        img_path_in_coco_json = img_info['file_name'] 
        
        # Cần ghép với thư mục gốc của ảnh đã sao chép.
        # img_path_in_coco_json có dạng 'split_name/image_name.jpg'
        # image_base_dir là /kaggle/working/faster_rcnn_coco_dataset/images
        # Cần tìm ra thư mục split cụ thể (train, val, test)
        
        split_folder_name = img_path_in_coco_json.split(os.sep)[0] # Lấy 'train', 'val', 'test'
        actual_image_path = os.path.join(image_base_dir, split_folder_name, os.path.basename(img_path_in_coco_json))

        if not os.path.exists(actual_image_path):
            print(f"Cảnh báo: Không tìm thấy ảnh tại {actual_image_path}. Bỏ qua.")
            continue

        img = Image.open(actual_image_path).convert("RGB")
        ann_ids = coco.getAnnIds(imgIds=img_info['id'])
        anns = coco.loadAnns(ann_ids)

        fig, ax = plt.subplots(1, figsize=(10, 8)) # Tăng kích thước hình
        ax.imshow(img)
        ax.set_title(f"Image ID: {img_info['id']}, File: {os.path.basename(img_info['file_name'])}")

        for ann in anns:
            [x, y, w, h] = ann['bbox']
            class_id = ann['category_id']
            # Đảm bảo class_id nằm trong danh sách category_id hợp lệ
            if coco.loadCats(class_id):
                class_name = coco.loadCats(class_id)[0]['name']
            else:
                class_name = f"Unknown Class ({class_id})" # Xử lý trường hợp không tìm thấy class_id

            rect = patches.Rectangle((x, y), w, h,
                                     linewidth=2, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
            plt.text(x, y - 5, class_name, color='red', fontsize=8, bbox=dict(facecolor='white', alpha=0.7, edgecolor='none', pad=1))
        plt.axis('off')
        plt.show()

# Gọi hàm kiểm tra trực quan cho mỗi tập dữ liệu
print("\nKiểm tra trực quan cho tập TRAIN:")
visualize_coco_boxes(TRAIN_JSON_PATH, OUTPUT_IMAGES_DIR, num_samples=3)

print("\nKiểm tra trực quan cho tập VAL:")
visualize_coco_boxes(VAL_JSON_PATH, OUTPUT_IMAGES_DIR, num_samples=3)

print("\nKiểm tra trực quan cho tập TEST:")
visualize_coco_boxes(TEST_JSON_PATH, OUTPUT_IMAGES_DIR, num_samples=3)

print("\n--- Kiểm tra trực quan Bounding Box hoàn tất ---")